# Project 2 - Community detection in an academic network
## Aim
<ol>
  <li> Load social graph</li>
  <li> Run community detection and centrality methods</li>
  <li> Visualize the network </li>
</ol>

## Tasks
<ol>
    <li> <strong>Load the dataset:</strong> Load the Author Network dataset provided at <a href:"https://aminer.org/lab-datasets/soinf/">https://aminer.org/lab-datasets/soinf/</a>
The graph consists of authors and coauthor relationships.
  </li>
    <li> <strong>Implementation:</strong> 
        <ol>
            <li> Implement Girvan-Newman clustering algorithm till 10th iteration level.</li>
            <li> Implement Pagerank algorithm.</li>
            <li> Implement Betweenness centrality measure</li>
        <strong>Use the previous implementation, perform the following tasks</strong>
            <li> Use Girvan-Newman algorithm to find clusters of authors</li>
            <li> Find the top-10 authors with highest betweenness centrality</li>      
        </ol>
    </li>
    <li> <strong>Visualization: </strong> 
        <ol>
            <li> Visualize the output of Girvan-Newman algorithm by coloring nodes according to their assigned groups</li>
            <li> Visualize the network and highlight the top 10 authors with the highest betweeness centrality and top 10 edges with the highest betweenness centrality</li>      
        </ol>
    </li>


# 1. Load the dataset

In [ ]:
# Import Modules
import networkx as nx
from pprint import pprint
import operator
import matplotlib.pyplot as plt
import numpy as np
import os
import re
import math
import random
from matplotlib.gridspec import GridSpec

# Utility class to pretty print dictionary
class DictTable(dict):
    # Overridden dict class which takes a dict in the form {'a': 2, 'b': 3},
    # and renders an HTML Table in IPython Notebook.
    def _repr_html_(self):
        html = ["<table width=100%>"]
        for key, value in iter(self.items()):
            html.append("<tr>")
            html.append("<td>{0}</td>".format(key))
            html.append("<td>{0}</td>".format(str(len(value))+" "+"graph[s] of size: "+str([len(x) for x in value])+" nodes"))
            html.append("</tr>")
        html.append("</table>")
        return ''.join(html)
print("> All necessary modules imported")

In [ ]:
# Load Data
DATASET = "./data"

def loadData(directoryPath):
    """Load the data of all files of the folder given in parameter."""
    files = os.listdir(directoryPath)
    # dictionary wrt to the following format: {topic1:[[network1],[network2]]}
    allGraphsOfEachTopic = {}
    for file in files:
        topic = ""
        if "T107" in file:
            topic = "Web Services"
        elif "T131" in file:
            topic = "Bayesian Networks/Belief function"
        elif "T144" in file:
            topic = "Web Mining/Information Fusion"
        elif "T145" in file:
            topic = "Semantic Web/Description Logics"
        elif "T162" in file:
            topic = "Machine Learning"
        elif "T16" in file:
            topic = "Data Mining/Association Rules"
        elif "T24" in file:
            topic = "Database Systems/XML Data"
        elif "T75" in file:
            topic = "Information Retrieval"
        else:
            topic = "Unknown"

        graphToBuild = nx.Graph()
        
        # for naming purposes
        subStart = file.find('sub')
        subEnd = file.find('.')
        graphToBuild.name = topic.replace('/', ' ').replace(' ', '') + '_{}'.format(file[subStart:subEnd])

        # constant
        VERTEX = 0
        EDGE = 1
        TRIANGLE = 2

        f = open(DATASET+"/"+file)

        # Vertices: Int "String" Int -> NodeID, personName, #papers
        # Edges: Int Int Int -> sourceNodeID, DestNodeID, #coauthoredPapers
        # Triangles: Int,Int,Int,Int -> NodeID1, NodeID2, NodeID3, #coauthoredPapers
        for line in f:
            if "*Vertices" in line:
                typeOfLine = VERTEX
            elif "*Edges" in line:
                typeOfLine = EDGE
            elif "*Triangles" in line:
                typeOfLine = TRIANGLE
            else:
                if typeOfLine == VERTEX:
                    graph_edge_list = []
                    
                    for index, s in enumerate(re.split('"', line)):
                        if (index == 1):
                            graph_edge_list.append(s)
                        else:
                            graph_edge_list.append(s.replace(' ','').replace('\n', ''))
                            
                    graphToBuild.add_node(graph_edge_list[0], name=graph_edge_list[1], nbpapers=graph_edge_list[2])
                elif typeOfLine == EDGE:
                    graph_edge_list = line.split()
                    graphToBuild.add_edge(graph_edge_list[0], graph_edge_list[1], coauthoredPapers=graph_edge_list[2])
                elif typeOfLine == TRIANGLE:
                    graph_edge_list = line.split(',')
                    graph_edge_list[3] = graph_edge_list[3].replace('\n', '')
                    graphToBuild.add_edge(graph_edge_list[0], graph_edge_list[1], coauthoredPapersTriangle=graph_edge_list[3])
                    graphToBuild.add_edge(graph_edge_list[0], graph_edge_list[2], coauthoredPapersTriangle=graph_edge_list[3])
                    graphToBuild.add_edge(graph_edge_list[1], graph_edge_list[2], coauthoredPapersTriangle=graph_edge_list[3])

        if topic in allGraphsOfEachTopic:
            allGraphsOfEachTopic[topic].append(graphToBuild)
        else:
            allGraphsOfEachTopic[topic] = [graphToBuild]

    return allGraphsOfEachTopic

allData = loadData(DATASET)
print("> All data loaded")
DictTable(allData)

# 2. Implementation
## 2A. Implementation of Girvan-Newman clustering till 10th iteration level

In [ ]:
def listAuthorsClusters(G):
    if not os.path.exists('./Output/GirvanNewman/ClustersTxt'):
        os.makedirs('./Output/GirvanNewman/ClustersTxt')
    
    connected_components = nx.connected_component_subgraphs(G)
    
    f = open('./Output/GirvanNewman/ClustersTxt/{}.txt'.format(G.name), 'w')
    for index, sg in enumerate(connected_components):
        f.write('######## Cluster {:2} #######\n'.format(index))
        for node in sg.nodes:
            f.write('> {}\n'.format(G.node[node]['name']))
        f.write('\n\n')
    f.close()


def girvanNewmanClustering(graph, nbIteration, visualization = False):
    print('Girvan-Newman on graph "{}", {} iterations'.format(graph.name, nbIteration))
    
    if visualization == True:
         # Draw first plot in figure and use variable cnt to handle positions
        fig = plt.figure(figsize=(50, 50))
        drawGraphs(graph, 1)
        cnt = 2
   
    
    # If there are no edge in the graph
    if len(list(graph.edges)) == 0:
        return "Empty graph"

    # While there are edges in the graph and that the number of iteration is biggger than 0
    while(len(list(graph.edges)) > 0 and nbIteration > 0):
        print("Remaining iterations: ", nbIteration)
        nbIteration = nbIteration - 1
        
        # Compute all shortest path between all nodes,transform this into a numpy array, put 0 at each element
        # under the diagonal of the matrix
        #allShortestPath = np.triu(np.asarray(list(nx.all_pairs_shortest_path(graph))),-2)
        
        # Delete all rows containging only 0
        #allShortestPath = allShortestPath[~np.all(allShortestPath == 0, axis=1)]
        print("test111")
        allShortestPaths = []
        for index, vi in enumerate(np.asarray(list(nx.all_pairs_shortest_path(graph)))):
            test = list(vi[1].values())
            test2= sorted(test, key=lambda x: int(x[-1]))[index:]
            for path in test2:
                allShortestPaths.append(path)
        print("test222", len(allShortestPaths))
        counter=0
        
        if len(allShortestPaths) ==0:
            break
        
        # Get all edges of the current graph
        edges = np.asarray(list(graph.edges))
        
        # Variable to store the highest edge and the centrality associated with it
        highestEdge = ""
        highestScore = -float('inf')
        
        for edge in edges:
            counter+=1
            print(counter)
            centrality = edgesBetweenessCentrality(allShortestPaths,edge)
            # Keep the edge with the highest centrality
            if centrality > highestScore:
                highestScore = centrality
                highestEdge = edge
                
        # Remove the edge with the highest centrality
        graph.remove_edge(highestEdge[0], highestEdge[1])
        
        if visualization == True:
            # Draw the graph with the removed edge into the figure
            drawGraphs(graph, cnt)
            cnt += 1
        
    
    if visualization == True:
        # Draw the original graph with colors to detect communities and save the figure
        print("Drawing...")
        drawColoredGraph(graph)
        print("Drawing done")
        if not os.path.exists('./Output/GirvanNewman/Figures'):
            os.makedirs('./Output/GirvanNewman/Figures')
    
        plt.savefig("./Output/GirvanNewman/Figures/{}.png".format(graph.name))
        print("> Figure saved")
    
    print('Saving clusters composition...')
    listAuthorsClusters(graph)
    print('> Clusters composition saved')                

## 2B. Implementation of Pagerank algorithm

In [ ]:
def pageRankCentrality(graph, alpha, beta):
    # Transposition of matrix
    adjacencyMatrix = nx.to_numpy_matrix(graph, weight='None')
    amTransposed = np.transpose(adjacencyMatrix)

    # Diagonal Matrix
    diagonalMatrix = np.zeros([adjacencyMatrix.shape[0], adjacencyMatrix.shape[1]])
    row, col = np.diag_indices(diagonalMatrix.shape[0])
    # Compute the values that have to be filled into the diagonal
    diagonalMatrix[row, col] = [1 / degree[1] for degree in list(graph.degree())]

    # Identity matrix
    identityMatrix = np.identity(adjacencyMatrix.shape[0])

    # Vector of ones
    ones = np.ones((adjacencyMatrix.shape[0], 1))
    pageRankCentrality = np.dot(beta * np.linalg.inv((identityMatrix - np.dot(alpha * amTransposed, diagonalMatrix))), ones)

    return pageRankCentrality



## 2C. Implementation of the betweenness centrality measure

In [ ]:
def edgesBetweenessCentrality(allShortestPath, edge):
    """computes the betweenness centrality of a given edge"""
    # Initialization of variables to compute centrality of each edge
    centrality=0
    nbPathsIncludingEdge=0
    counterShortestPath=0
            
    # For all shortestPath, count the number of them that contain the current edge 
    for vi in allShortestPath:
        nbPathsIncludingEdge += edgeIsInPath([edge[0], edge[1]], vi)
        counterShortestPath+=1
            
    # Centrality = Number of shortest paths including the edge / total number of shortest paths       
    centrality += nbPathsIncludingEdge / counterShortestPath
    return centrality

def edgeIsInPath(edge, path):
    """checks if an edge is contained in a path of an undirected graph (the path is a list of nodes)"""
    edge1= [edge[0],edge[1]]
    edge2= [edge[1],edge[0]]
    if all(i in path for i in edge1) or all(i in path for i in edge2):
        return 1
    else:
        return 0

    
def nodeIsInPath(node, path):
    """checks if an edge is contained in a path of an undirected graph (the path is a list of nodes)"""
    if node in path:
        return 1
    else:
        return 0
    
def nodesBetweennessCentrality(listOfAllShortestPath, node, graph):
    """betweenness centrality of a given node: used in task 2E"""
    centrality = 0
    nbPathsIncludingNode=0
    counterShortestPath=0
    
    for vi in listOfAllShortestPath:
        nbPathsIncludingNode += nodeIsInPath(node, vi)
        counterShortestPath+=1
        
    centrality = nbPathsIncludingNode/counterShortestPath
    return centrality

## 2D. Use Girvan-Newman algorithm to find clusters of authors

In [ ]:
# runs Girvan-Newman for every graph in the dataset
for topic, listOfGraphs in allData.items():
    print('>>> Topic "{}""'.format(topic))
    for graph in listOfGraphs:
        print('> Girvan-Newman clustering for {}'.format(graph.name))
        girvanNewmanClustering(graph.copy(), 10)

## 2E. Find the top-10 authors with highest betweenness centrality

In [ ]:
# prints the top-10 authors for each graph of the dataset        
for topic, listOfGraphs in allData.items():
    print('Topic "{}""'.format(topic))
    for graph in listOfGraphs:
        results = []
        allShortestPaths = []
        for index, vi in enumerate(np.asarray(list(nx.all_pairs_shortest_path(graph)))):
            test = list(vi[1].values())
            test2= sorted(test, key=lambda x: int(x[-1]))[index+1:]
            for path in test2:
                allShortestPaths.append(path)
        for node in graph.nodes:
            results.append( (nodesBetweennessCentrality(allShortestPaths, node, graph), graph.node[node]['name']) )
        print('Top-10 authors for {}'.format(graph.name))
        for tuple in sorted(results, reverse=True)[:10]:
            print('>>> {}'.format(tuple))

In [ ]:
# Example with a single graph: TO DELETE FOR THE FINALE VERSION BUT IS CURRENTLY A PROOF THAT THE ALGORITHM WORKS 
                            #AS IS PRODUCES THE SAME RESULT THAN THE NETWORKX FUNCTION 
graph = allData["Web Mining/Information Fusion"][2].copy()
print("Here are the results given by networkX \n",nx.betweenness_centrality(graph,endpoints=True))
results = []

allShortestPaths = []
for index, vi in enumerate(np.asarray(list(nx.all_pairs_shortest_path(graph)))):
    test = list(vi[1].values())
    test2= sorted(test, key=lambda x: int(x[-1]))[index+1:]
    for path in test2:
        allShortestPaths.append(path)
        
for node in graph.nodes:
    results.append( (nodesBetweennessCentrality(allShortestPaths, node, graph), graph.node[node]['name']) )
print('Top-10 authors for {}'.format(graph.name))
for tuple in sorted(results, reverse=True)[:10]:
    print('>>> {}'.format(tuple))


# 3. Visualization

## 3A. Visualize the output of Girvan-Newman algorithm by coloring nodes according to their assigned groups

In [ ]:
def drawGraph(G):
    """draws the given graph and displays the labels of each edge"""
    pos = nx.spring_layout(G)
    nx.draw(G, pos, with_labels=True)
    nx.draw_networkx_edge_labels(G, pos)
    plt.show()

def drawGraphs(G, cnt):
    """Draw all different subgraphs on the same picture"""
    test = plt.subplot(4, 3, cnt)
    if (cnt == 1):
        test.title.set_text('Initial state of the graph')
    else:
        test.title.set_text('State of the graph at iteration: {}, number of communities: {}'.format(cnt-1, nx.number_connected_components(G)))
    test.set_yticklabels([])
    test.set_xticklabels([])
    pos = nx.spring_layout(G)
    nx.draw_networkx(G, pos)
    nx.draw_networkx_edge_labels(G, pos)

def drawColoredGraph(G):
    """# draws a graph where communities are drawn using different colors"""
    test = plt.subplot(4,3,12)
    test.set_yticklabels([])
    test.set_xticklabels([])
    test.title.set_text('Resulting communities ({})'.format(nx.number_connected_components(G)))
    pos = nx.spring_layout(G)
    connected_components = nx.connected_component_subgraphs(G)
    for index, sg in enumerate(connected_components):
        r = lambda: random.randint(0,255)
        randomColor =('#%02X%02X%02X' % (r(),r(),r()))
        nx.draw_networkx(sg, pos = pos, node_color = str(randomColor), with_labels= False, edgelist=[])
        

In [ ]:
# runs Girvan-Newman for every graph in the dataset and visualize the output
for topic, listOfGraphs in allData.items():
    print('>>> Topic "{}""'.format(topic))
    for graph in listOfGraphs:
        print('> Girvan-Newman clustering for {}'.format(graph.name))
        girvanNewmanClustering(graph.copy(), 10, visualization=True)

## 3B. Visualize the network and highlight the top 10 authors with the highest betweenness centrality and top 10 edges with the highest betweenness centrality